In [1]:
# Cora dataset
from torch_geometric.datasets import Planetoid
 
# Import dataset from PyTorch Geometric
dataset = Planetoid(root=".", name="Cora") 
data = dataset[0]
# Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

/home/ubuntu/anaconda3/envs/bgefinetune/lib/python3.10/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /home/ubuntu/anaconda3/envs/bgefinetune/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/ubuntu/anaconda3/envs/bgefinetune/lib/python3.10/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /home/ubuntu/anaconda3/envs/bgefinetune/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
def accuracy(preds, labels):
    """Compute the accuracy of predictions."""
    correct = (preds == labels).sum().item()
    total = labels.size(0)
    return correct / total

In [ ]:
# Basic GNN
from torch_geometric.utils import to_dense_adj

class GNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        
        # bias를 포함하지 않는 basic linear transformation
        self.linear = nn.Linear(dim_in, dim_out, bias=False)
 
    def forward(self, x, edge_index):
        # adj matrix
        adjacency = to_dense_adj(data.edge_index)[0]
        adjacency += torch.eye(len(adjacency))
        # (1) Linear trasnformation
        x = self.linear(x) # hidden layer (X*W.T)
        
        # (2) 메세지 전달, Multiplication with the adjacency matrix A
        x = torch.sparse.mm(adjacency, x) # H = A.T (X*W.T)
        return x

In [8]:
class GNN(torch.nn.Module):
    """Basic Graph Neural Network"""
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gnn1 = GNNLayer(dim_in, dim_h)
        self.gnn2 = GNNLayer(dim_h, dim_out)
 
    def forward(self, x, edge_index):
        h = self.gnn1(x, edge_index)
        h = torch.relu(h)
        h = self.gnn2(h, edge_index)
        return F.log_softmax(h, dim=1)
 
    def fit(self, data, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(),
                                      lr=0.01,
                                      weight_decay=5e-4)
 
        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            
            out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1),
                          data.y[data.train_mask])
            loss.backward()
            optimizer.step()
 
            if(epoch % 20 == 0):
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1),
                                  data.y[data.val_mask])
                print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc:'
                      f' {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | '
                      f'Val Acc: {val_acc*100:.2f}%')
 
    @torch.no_grad()
    def test(self, data):
        self.eval()
        out = self(data.x, data.edge_index)
        acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
        return acc

In [9]:
# Create theGNN model
gnn = GNN(dataset.num_features, 16, dataset.num_classes)
print(gnn)
 
# Train
gnn.fit(data, epochs=100)
 
# Test
acc = gnn.test(data)
print(f'\nGNN test accuracy: {acc*100:.2f}%')

GNN(
  (gnn1): GNNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=False)
  )
  (gnn2): GNNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=False)
  )
)
Epoch   0 | Train Loss: 2.021 | Train Acc: 12.86% | Val Loss: 1.97 | Val Acc: 14.80%
Epoch  20 | Train Loss: 0.042 | Train Acc: 100.00% | Val Loss: 1.80 | Val Acc: 73.60%
Epoch  40 | Train Loss: 0.003 | Train Acc: 100.00% | Val Loss: 2.31 | Val Acc: 75.00%
Epoch  60 | Train Loss: 0.001 | Train Acc: 100.00% | Val Loss: 2.42 | Val Acc: 74.60%
Epoch  80 | Train Loss: 0.001 | Train Acc: 100.00% | Val Loss: 2.39 | Val Acc: 74.80%
Epoch 100 | Train Loss: 0.001 | Train Acc: 100.00% | Val Loss: 2.31 | Val Acc: 74.40%

GNN test accuracy: 75.70%
